In [ ]:
import numpy as np
import skvideo.io
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.measure import label
from skimage.morphology import remove_small_objects, erosion, dilation, disk

from trackbox import *

### 0. Load the video
* Change `filename = "name_of_video.wmv"` to the name of your own video.
* Change `down_sample=1` to `n` to speed-up the video by `n` times.
* Change `start` and `end` to the MM:SS format (e.g., `3:11`) to trim the video.

In [ ]:
filename = "name_of_video.wmv"
video, video_gray = load_video(filename, down_sample=1, start='0:09', end='4:09')
show_rand_imgs(video, 3)
show_rand_imgs(video_gray, 3, 'gray')

### 1. Find valid region in the field-of-view (FoV)

In [ ]:
valid_region = find_valid_region(video_gray.mean(0))

### 2. Process and save the video

In [ ]:
all_centers = []
center_video = np.zeros(video_gray.shape, dtype=np.uint8)
for i in list(range(video_gray.shape[0])):
    center = segment_image(video_gray[i], show_imgs=False, valid_region=valid_region)
    if len(center)==2:
        center_video[i, center[0], center[1]] = 1
        all_centers.append(center)
save_video(video, video_gray, center_video)

In [ ]:
temp = np.ones(video_gray[50].shape, dtype=np.uint8)*255
temp[0,0] = 0
plt.imshow(temp, cmap='gray')
for j in range(len(all_centers)-1):
    # print(j, all_centers[j])
    plt.plot([all_centers[j][1], 
              all_centers[j+1][1]], 
             [all_centers[j][0],
              all_centers[j+1][0]], 'r', alpha=0.3, linewidth=2)